In [2]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=('PEPE',))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")

22:33:35: Main    : before creating thread
22:33:35: Main    : before running thread
22:33:35: Thread PEPE: starting
22:33:35: Main    : wait for the thread to finish
22:33:35: Main    : all done


22:33:37: Thread PEPE: finishing


In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/DataSet_Titanic.csv")

In [5]:
df

,Sobreviviente,Clase,Genero,Edad,HermEsp,PadHij
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0
...,...,...,...,...,...,...
709,0,3,1,39.0,0,5
710,0,2,0,27.0,0,0
711,1,1,1,19.0,0,0
712,1,1,0,26.0,0,0


In [30]:
p = 'clase-0'

p.replace('clase-', '')

'0'

In [5]:
qualitative_variables_form = []
for column in df.drop(columns='Sobreviviente'):
    if len(set(df[column])) < 5:
        qualitative_variables_form.append({
            'name': column,
            'variables': list(set(df[column]))
        })
print(qualitative_variables_form)

[{'name': 'Clase', 'variables': [1, 2, 3]}, {'name': 'Genero', 'variables': [0, 1]}]


In [15]:
from sklearn.model_selection import train_test_split

data = df.drop(columns="Sobreviviente")
x_train, x_test, y_train, y_test = train_test_split(data,
                                                    df["Sobreviviente"], test_size=0.2,
                                                    random_state=123)


In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [11]:
possible_targets = list(set(df.columns) - set(clf.feature_names_in_))

In [12]:
possible_targets

['Sobreviviente']

In [4]:
import joblib
model = joblib.dump(clf,"D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

NameError: name 'clf' is not defined

In [ ]:
print(data.columns)

In [40]:
def get_rules(model, q_variables, q_variables_values, features, class_names, target):
    tree_ = model
    feature_name = [
        features[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    cont = 1

    def get_simplified_route(route):
        simplified_route = list(route)
        watched_classes_plus = []
        watched_classes_less = []
        done = False
        iteration = 0
        while (not done):
            analyzed_rule = {}
            analyzed_item = ""
            analyzed_sign = ""
            analyzed_value = []
            analyzed_type = ""
            for i in range(len(route)):
                if (iteration + i) < len(route):
                    actual = route[i + iteration]
                    if actual["item"] not in DT["q_variables"] and (analyzed_type == "number" or not analyzed_type):
                        if not analyzed_item:
                            analyzed_rule = actual
                            analyzed_type = "number"
                            analyzed_item = actual["item"]
                            analyzed_value = actual["value"]
                            if analyzed_item not in watched_classes_plus and actual["sign"] == " > ":
                                watched_classes_plus.append(analyzed_item)
                                analyzed_sign = " > "
                            elif analyzed_item not in watched_classes_less and actual["sign"] == " <= ":
                                watched_classes_less.append(analyzed_item)
                                analyzed_sign = " <= "
                        elif actual["item"] == analyzed_item and actual["sign"] == analyzed_sign:
                            try:
                                if analyzed_sign == " > ":
                                    if actual["value"] > analyzed_value:
                                        simplified_route.remove(analyzed_rule)
                                        analyzed_value = actual["value"]
                                        analyzed_rule = actual
                                    elif actual["value"] <= analyzed_value:
                                        simplified_route.remove(actual)
                                elif analyzed_sign == " <= ":
                                    if actual["value"] <= analyzed_value:
                                        simplified_route.remove(analyzed_rule)
                                        analyzed_value = actual["value"]
                                        analyzed_rule = actual
                                    elif actual["value"] > analyzed_value:
                                        simplified_route.remove(actual)
                            except Exception as e:
                                str(e)
                                break
                    elif analyzed_type == "object" or not analyzed_type:
                        if not analyzed_item:
                            analyzed_rule = actual
                            analyzed_type = "object"
                            analyzed_sign = "es :"
                            analyzed_item = actual["item"]
                            analyzed_value = actual["value"]
                        elif actual["item"] == analyzed_item:
                            try:
                                if all(val in analyzed_value for val in actual["value"]):
                                    simplified_route.remove(analyzed_rule)
                                    analyzed_value = actual["value"]
                                elif all(analyzed_val in actual["value"] for analyzed_val in analyzed_value):
                                    simplified_route.remove(actual)
                            except Exception as e:
                                str(e)
                                break
                else:
                    break
            iteration += 1
            if iteration >= len(route):
                done = True
        return simplified_route

    def get_rule(name, threshold, bigger):
        rule = [{
            "item": name,
            "sign": "",
            "value": []
        }]
        if name in q_variables:
            new_args = []
            first = True
            # condition = "es : "
            rule[0]["sign"] = "es: "
            for variables in q_variables_values:
                if variables["column"] == name:
                    for values in variables["values"]:
                        if (not bigger and values["previous_value"] <= threshold) or (
                                bigger and values["previous_value"] > threshold):
                            new_args.append(values["new_value"])
            rule[0]["value"] = new_args
        else:
            if not bigger:
                rule[0]["sign"] = " <= "
            else:
                rule[0]["sign"] = " > "
            rule[0]["value"] = [f"{np.round(threshold, 3)}"]

        return rule

    def get_text_route(route):
        text_route = []
        for val in route:
            if val["item"] not in q_variables:
                text_route += [(f"({val['item']} {val['sign']} {val['value'][0]})")]
            else:
                values = ""
                first = True

                for i in range(len(val['value'])):
                    if first:
                        values += f"{val['value'][i]}"
                        first = False
                    else:
                        values += f" o {val['value'][i]}"
                text_route += [(f"{val['item']} {val['sign']} {values}")]
        return text_route

    def recurse(node, route, routes):

        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path_rules, right_path_rules = list(route), list(route)
            left_path_rules += get_rule(name, threshold, bigger=False)
            recurse(tree_.children_left[node], left_path_rules, routes)
            right_path_rules += get_rule(name, threshold, bigger=True)
            recurse(tree_.children_right[node], right_path_rules, routes)
        else:
            text_route_original = get_text_route(route)
            text_route_original += [(tree_.value[node], tree_.n_node_samples[node])]
            simplified_route = get_simplified_route(route)
            text_route = get_text_route(simplified_route)
            text_route += [(tree_.value[node], tree_.n_node_samples[node])]
            routes += [text_route]

    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]

    rules = []
    for path in paths:
        rule = "Si "

        for p in path[:-1]:
            if rule != "Si ":
                rule += " y "
            rule += str(p)
        rule += " entonces "
        if class_names is None:
            rule += "respuesta: " + str(np.round(path[-1][0][0][0], 3))
        else:
            classes = path[-1][0][0]
            position = np.argmax(classes)
            rule += f"{target}: {class_names[position]} (con probabilidad: " \
                    f"{np.round(100.0 * classes[position] / np.sum(classes), 2)}%)"
        rule += f" | basado en {path[-1][1]:,} muestras"
        rules += [f"Regla {cont}: {rule}"]
        cont += 1

    return rules

In [ ]:
rules = get_rules(DT, DT["model"].tree_, DT["features"], DT["class_names"], DT["target"])
for r in rules:
    print(r)
    print()

In [ ]:
from dtreeviz import model


def graph_tree(DT):
    viz = model(
        DT["model"],
        X_train=x_train,
        y_train=y_train,
        feature_names=list(DT["features"]),
        class_names=DT["class_names"]
    )
    try:
        svg = open(viz.view().save_svg(), "rb").read()
        encoded = base64.b64encode(svg)
        svg_encoded = "data:image/svg+xml;base64,{}".format(encoded.decode())
        return svg_encoded, viz
    except Exception as e:
        print(e)
        return "", viz

In [ ]:
x, viz = graph_tree(DT)

In [ ]:
viz.view().show()

In [ ]:
print(list(DT["features"]))

In [10]:
from model.model import ClassificationTrainedModel

In [9]:
import joblib
from sklearn.ensemble import RandomForestClassifier

random_forest_model: RandomForestClassifier = joblib.load(
    "D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

In [8]:
q_variables_values_list = [
    {
        "column_name": "Genero",
        "variables": [
            {
                "old_value": 0,
                "new_value": "Masculino"
            },
            {
                "old_value": 1,
                "new_value": "Femenino"
            },
        ]
    },
    {
        "column_name": "Clase",
        "variables": [
            {
                "old_value": 1,
                "new_value": "Primera Clase"
            },
            {
                "old_value": 2,
                "new_value": "Segunda Clase"
            },
            {
                "old_value": 3,
                "new_value": "Tercera Clase"
            },
        ]
    }
]

target_description = [
    {
        "column_name": "Sobreviviente",
        "variables": [
            {
                "old_value": 0,
                "new_value": "Muere"
            },
            {
                "old_value": 1,
                "new_value": "Vive"
            },
        ]
    }
]

In [11]:
explainer_classifier = ClassificationTrainedModel(name="Titanic", df=df, predictors_description="features_description",
                                                  target="Sobreviviente", test_size=0.2, random_state=123,
                                                  model=random_forest_model, model_description="",
                                                  target_description=target_description[0],
                                                  q_variables_values_list=q_variables_values_list)


{'column_name': 'Genero', 'variables': [{'old_value': 0, 'new_value': 'Masculino'}, {'old_value': 1, 'new_value': 'Femenino'}]}
{'column_name': 'Clase', 'variables': [{'old_value': 1, 'new_value': 'Primera Clase'}, {'old_value': 2, 'new_value': 'Segunda Clase'}, {'old_value': 3, 'new_value': 'Tercera Clase'}]}
Creating data_table...


KeyError: 'column'

In [57]:
from sklearn import tree
from sklearn.tree import _tree
import base64

DT = {
    # "model": tree.DecisionTreeClassifier(max_depth=2).fit(x_train, y_train),
    "model": random_forest_model.estimators_[25],
    "features": data.columns,
    "class_names": ["Muere", "Vive"],
    "target": "Sobreviviente",
    "q_variables": ["Genero", "Clase"],
    "q_variables_values": q_variables_values_list,
}

In [58]:
# rules = get_rules(DT, DT["model"].tree_, DT["features"], DT["class_names"], DT["target"])
rules = get_rules(model=DT["model"].tree_, q_variables=DT["q_variables"], q_variables_values=DT["q_variables_values"], features=DT["features"], class_names=DT["class_names"], target=DT["target"])
for r in rules:
    print(r)
print()

Regla 1: Si (PadHij  <=  0.5) y Genero es:  Masculino entonces Sobreviviente: Muere (con probabilidad: 83.16%) | basado en 188 muestras
Regla 2: Si (PadHij  <=  0.5) y Genero es:  Femenino entonces Sobreviviente: Vive (con probabilidad: 72.88%) | basado en 78 muestras
Regla 3: Si (PadHij  >  0.5) y Genero es:  Masculino entonces Sobreviviente: Muere (con probabilidad: 66.67%) | basado en 47 muestras
Regla 4: Si (PadHij  >  0.5) y Genero es:  Femenino entonces Sobreviviente: Vive (con probabilidad: 81.94%) | basado en 46 muestras



ROC_CURVE TESTING

In [7]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

In [8]:
options = [
    {
        'label': "Label 1"
    },
    {
        'label': "Label 2"
    },
]

y_pred = random_forest_model.predict_proba(x_test)

NameError: name 'random_forest_model' is not defined

In [11]:
def create_curve(y_scores, y_true, options):
        # One hot encode the labels in order to plot them
        # y_onehot = pd.get_dummies(y_true)
        # print(y_onehot)

        data = []
        trace1 = go.Scatter(x=[0, 1], y=[0, 1],
                            mode='lines',
                            line=dict(dash='dash'),
                            showlegend=False

                            )

        data.append(trace1)
        cont = 0
        for i in range(y_scores.shape[1]):
            y_score = y_scores[:, i]

            fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=i)
            auc_score = auc(fpr, tpr)
            
            print("fpr: ")
            print(fpr)
            print("tpr: ")
            print(tpr)

            name = f"{options[cont]['label']} (AUC={auc_score:.2f})"
            trace2 = go.Scatter(x=fpr, y=tpr,
                                name=name,
                                mode='lines')
            data.append(trace2)
            cont += 1

        layout = go.Layout(
            title='ROC-AUC curva',
            yaxis=dict(title='Tasa de Positivos'),
            xaxis=dict(title='Tasa de Falsos Positivos')
        )

        fig = go.Figure(data=data, layout=layout)

        return fig

In [12]:
create_curve(y_scores=y_pred, y_true=y_test, options=options)

fpr: 
[0.         0.08928571 0.08928571 0.10714286 0.14285714 0.14285714
 0.16071429 0.16071429 0.16071429 0.17857143 0.17857143 0.19642857
 0.19642857 0.19642857 0.19642857 0.25       0.25       0.26785714
 0.26785714 0.28571429 0.28571429 0.30357143 0.35714286 0.35714286
 0.39285714 0.39285714 0.39285714 0.41071429 0.41071429 0.42857143
 0.42857143 0.5        0.57142857 0.57142857 0.76785714 0.83928571
 0.85714286 0.92857143 1.        ]
tpr: 
[0.         0.10344828 0.12643678 0.16091954 0.35632184 0.36781609
 0.40229885 0.42528736 0.44827586 0.52873563 0.55172414 0.5862069
 0.65517241 0.67816092 0.68965517 0.71264368 0.73563218 0.73563218
 0.79310345 0.79310345 0.81609195 0.82758621 0.82758621 0.87356322
 0.88505747 0.89655172 0.91954023 0.91954023 0.95402299 0.95402299
 0.98850575 0.98850575 0.98850575 1.         1.         1.
 1.         1.         1.        ]
fpr: 
[0.         0.         0.         0.         0.         0.
 0.         0.01149425 0.01149425 0.01149425 0.04597701 0.

CONFUSION MATRIX

In [ ]:
from sklearn import metrics
import plotly.express as px

In [ ]:



cm = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred)

fig = px.imshow(img=cm,
                        title="MATRIZ DE CONFUSION",
                        labels=dict(x="PREDICCIONES", y="VALORES REALES", color="CANTIDAD"),
                        x=class_names,
                        y=class_names,
                        text_auto=True,
                        color_continuous_scale="Blues",
                        )

In [4]:
import dash
import numpy as np
import pandas as pd

from dash import dash_table

# Create a pd.MultiIndex from the combination of t & m:
t = ["1M", "3M", "6M", "1Y"]
m = ["IV", "RV", "Spread"]

arrays = np.array(sorted([[b, a] for a in m for b in t]))

df = pd.DataFrame(
    sorted(arrays, key=lambda x: (x[0][1], x[0][0])), columns=["Tenor", None]
)

print(df)

index = pd.MultiIndex.from_frame(df)

# Create a mock df using random np floats, specifying the columns
# with the previously created pd.MultiIndex and the "index" here
# as the row labels
df2 = pd.DataFrame(
    np.around(np.random.randn(3, 12), decimals=2),
    index=["EURUSD", "GBPUSD", "USDJPY"],
    columns=index,
)

# Dash app
app = dash.Dash(__name__)

"""For getting the columns fed correctly to dash_table,
a two-row multi-header can be created by supplying 
the 'name' key of `DataTable.columns` with an array.

The trick then is to create unique IDs, which requires 
manipulation of the data into a list of dictionaries 
where each cell value's key is the artificially created
concatenated string (I just combined them; e.g., "1M_IV" 
is one of the unique keys, and so on).

Note: the use of '**' is a useful Python3+ way to merge 
dicts during list/dict comprehensions. This is necessary
for including the true index 'Ccy Pair' as key,value 
pairs in addition to a dict comprehension through the data.
Thus I needed to also transpose the df..."""
 
app.layout = dash_table.DataTable(
    id="table",
    columns=[{"name": ["Tenor", "Ccy Pair"], "id": "Ccy Pair"}]
    + [{"name": [x1, x2], "id": f"{x1}_{x2}"} for x1, x2 in df2.columns],
    data=[
        {
            **{"Ccy Pair": df2.index[n]},
            **{f"{x1}_{x2}": y for (x1, x2), y in data},
        }
        for (n, data) in [
            *enumerate([list(x.items()) for x in df2.T.to_dict().values()])
        ]
    ],
    merge_duplicate_headers=True,   # ← here's the main 🪄 😉 
    # Optional interactivity parameters＊:
    editable=True,
    filter_action="native",
    sort_action="native",
    sort_mode="multi",
    column_selectable="single",
    row_selectable="multi",
    row_deletable=True,
    selected_columns=[],
    selected_rows=[],
    page_action="native",
    page_current= 0,
    page_size= 10,
)

if __name__ == "__main__":
    app.run_server(debug=True, dev_tools_hot_reload=True)


   Tenor    None
0     1M      IV
1     1M      RV
2     1M  Spread
3     3M      IV
4     3M      RV
5     3M  Spread
6     6M      IV
7     6M      RV
8     6M  Spread
9     1Y      IV
10    1Y      RV
11    1Y  Spread


In [6]:
import dash_bootstrap_components as dbc
import dash

from dash import html

table1_header = [html.Thead(html.Tr([html.Th("First Name"), html.Th("Last Name")]))]


rrow1 = html.Tr([html.Td("Arthur"), html.Td("Dent")])

rrow2 = html.Tr([html.Td("Ford"), html.Td("Prefect")])

rrow3 = html.Tr([html.Td("Zaphod"), html.Td("Beeblebrox")])

rrow4 = html.Tr([html.Td("Trillian"), html.Td("Astra")])


table1_body = [html.Tbody([rrow1, rrow2, rrow3, rrow4])]


t1_layout = dbc.Table(table1_body, bordered=True)

table_header = [html.Thead(html.Tr([html.Th("First Name"), html.Th("Last Name")]))]

row1 = html.Tr([html.Td(t1_layout), html.Td("Dent")])

row2 = html.Tr([html.Td(t1_layout), html.Td("Prefect")])

row3 = html.Tr([html.Td(t1_layout), html.Td("Beeblebrox")])

row4 = html.Tr([html.Td(t1_layout), html.Td("Astra")])


table_body = [html.Tbody([row1, row2, row3, row4])]


app = dash.Dash(__name__)


app.layout = dbc.Table(table_header + table_body, bordered=True)



if __name__ == "__main__":

    app.run_server(debug=True, dev_tools_hot_reload=True)

In [3]:
import dash
from dash import dash_table
import pandas as pd

app = dash.Dash(__name__)

rrow1 = html.Tr([html.Td("Arthur"), html.Td("Dent")])
rrow2 = html.Tr([html.Td("Ford"), html.Td("Prefect")])
rrow3 = html.Tr([html.Td("Zaphod"), html.Td("Beeblebrox")])
rrow4 = html.Tr([html.Td("Trillian"), html.Td("Astra")])

table1_body = [html.Tbody([rrow1, rrow2, rrow3, rrow4])]

t1_layout = dbc.Table(table1_body, bordered=True)

# Create a sample dataframe
df = pd.DataFrame({
    'Column 1': [t1_layout, 'Value 2', 'Value 3'],
    'Column 2': [t1_layout, 'Value 5', 'Value 6'],
    'Column 3': [t1_layout, 'Value 5', 'Value 6']
})


# Create a column with a nested table
columns = [
    {'id': 'Column 1', 'name': 'Column 1'},
    {'id': 'Column 2', 'name': 'Column 2'},
    {'id': 'Column 3', 'name': 'Column 3', 'presentation': 'markdown'},
]


app.layout = dash_table.DataTable(
    id='datatable',
    columns=columns,
    data=df.to_dict('records'),
    style_data={'border': '1px solid blue'},
    style_header={'border': '1px solid pink'}
)


if __name__ == '__main__':
    app.run_server(debug=True)